In [1]:
from google.colab import drive
drive.mount('/content/drive/')
from pandas import DataFrame, Series
import pandas as pd
import glob
import re
from tqdm import tqdm
import torchaudio
import numpy as np
import itertools
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import pickle
import os
import glob
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input 
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from keras.models import load_model
from IPython.display import Audio
from IPython.core.display import display
import time

!pip install nussl
!pip install common

Mounted at /content/drive/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.9/239.9 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.5/266.5 KB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 KB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.3/203.3 KB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 KB 21.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.6/155.6 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 KB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.5/963.5 KB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

### import fine

In [2]:
fine,sr = librosa.load("/content/drive/MyDrive/Startup_Hackathon_Jarvis/fine_data/fine.wav", sr = 44100)
yj_fine,sr = librosa.load("/content/drive/MyDrive/Startup_Hackathon_Jarvis/fine_data/yj_fine.wav", sr = 44100)
migyo_fine,sr = librosa.load("/content/drive/MyDrive/Startup_Hackathon_Jarvis/fine_data/migyo_fine.wav", sr = 44100)

In [ ]:
Audio(data = fine, rate = sr)

# ✅ nussl mr 제거
https://source-separation.github.io/tutorial/first_steps/repetition.html#applying-the-masks


In [ ]:
import nussl
signal = nussl.AudioSignal(
    audio_data_array=fine, sample_rate=44100)


    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


In [ ]:
import nussl
short_signal = nussl.AudioSignal(
    audio_data_array=fine[int(5*44100) : int(10*44100)], sample_rate=44100)

In [ ]:
import nussl
signal_us = nussl.AudioSignal(
    audio_data_array=fine[int(5*0.4*44100) : int(6*0.4*44100)], sample_rate=44100)
repet = nussl.separation.primitive.FT2D(signal_us)
repet_bg, repet_fg = repet()

### 1) REPET

In [ ]:
t1 = time.time()
repet = nussl.separation.primitive.Repet(short_signal)
repet_bg, repet_fg = repet()
t2 = time.time()
print(f"Repet 짧은 오디오 처리 시간 : {t2 -t1}S")

Repet 짧은 오디오 처리 시간 : 0.28232550621032715S


In [ ]:
t1 = time.time()
repet = nussl.separation.primitive.Repet(signal)
repet_bg, repet_fg = repet()
t2 = time.time()
print(f"Repet 처리 시간 : {t2 -t1}S")

Repet 처리 시간 : 17.943342924118042


In [ ]:
Audio(data = repet_fg.audio_data, rate = 44100)

Output hidden; open in https://colab.research.google.com to view.

### 2) REPET-SIM

In [ ]:
t1 = time.time()
repet = nussl.separation.primitive.RepetSim(short_signal)
repet_bg, repet_fg = repet()
t2 = time.time()
print(f"RepetSim 짧은 오디오 처리 시간 : {t2 -t1}S")

RepetSim 짧은 오디오 처리 시간 : 0.5291290283203125S


In [ ]:
t1 = time.time()
repet_sim = nussl.separation.primitive.RepetSim(signal)
rsim_bg, rsim_fg = repet_sim()
t2 = time.time()
print(f"RepetSim 처리 시간 : {t2 -t1}S")

RepetSim 처리 시간 : 254.87003684043884S


In [ ]:
Audio(data = rsim_fg.audio_data, rate = 44100)

Output hidden; open in https://colab.research.google.com to view.

### 3) 2DFT

In [ ]:
t1 = time.time()
repet = nussl.separation.primitive.FT2D(short_signal)
repet_bg, repet_fg = repet()
t2 = time.time()
print(f"FT2D 짧은 오디오 처리 시간 : {t2 -t1}S")

FT2D 짧은 오디오 처리 시간 : 1.5391595363616943S


In [ ]:
t1 = time.time()
ft2d = nussl.separation.primitive.FT2D(signal)
ft2d_bg, ft2d_fg = ft2d()
t2 = time.time()
print(f"FT2D 처리 시간 : {t2 -t1}S")

FT2D 처리 시간 : 20.444836854934692S


In [ ]:
Audio(data = ft2d_fg.audio_data, rate = 44100)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
ft2d_fg.audio_data.shape

(1, 9251329)

In [ ]:
t1 = time.time()
ft2d = nussl.separation.primitive.FT2D(signal)
ft2d_bg, ft2d_fg = ft2d()
t2 = time.time()
print(f"FT2D 처리 시간 : {t2 -t1}S")

# ✔ 자동화

In [ ]:
import librosa
label = {0:"belt", 1:"straight", 2:"vibrato", 3:"vocal_fry"}
model = load_model("/content/drive/MyDrive/Startup_Hackathon_Jarvis/model_weight/custom_vgg16.h5", compile=False)
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=1e-4), metrics=['accuracy'])

In [ ]:
idx = 0
info = []
data = ft2d_fg.audio_data
techniques = []
while (idx+1)*0.4 < (9251329/44100):
  x, _ = librosa.magphase(data[0][int(idx*0.4*44100) : int((idx+1)*0.4*44100)])
  ret = librosa.feature.melspectrogram(y=x, sr=44100)
  amp = librosa.amplitude_to_db(np.abs(ret))
  amp_re = tf.reshape(amp, [-1, amp.shape[0], amp.shape[1], 1])
  result = model.predict(amp_re)
  info.append({"idx" : idx, "predict" : np.argmax(result)})

  if idx >= 2: 
    if info[idx]["predict"] == info[idx-1]["predict"] and info[idx]["predict"] == info[idx-2]["predict"] and info[idx]["predict"] != 1:
      print(label.get(np.argmax(result)))
      techniques.append({"idx" : idx, "label" : label.get(np.argmax(result))})
  idx+=1


1/1 [==============================] - 0s 103ms/step
vibrato
1/1 [==============================] - 0s 112ms/step
vibrato
1/1 [==============================] - 0s 84ms/step
vibrato
1/1 [==============================] - 0s 81ms/step
vibrato
1/1 [==============================] - 0s 84ms/step
vibrato
1/1 [==============================] - 0s 81ms/step
vibrato
1/1 [==============================] - 0s 108ms/step
vibrato
1/1 [==============================] - 0s 99ms/step
vibrato
1/1 [==============================] - 0s 109ms/step
vibrato
1/1 [==============================] - 0s 105ms/step
vibrato
1/1 [==============================] - 0s 74ms/step
vibrato
1/1 [==============================] - 0s 96ms/step
vibrato
1/1 [==============================] - 0s 81ms/step
vibrato
1/1 [==============================] - 0s 81ms/step
vibrato
1/1 [==============================] - 0s 84ms/step
vibrato
1/1 [==============================] - 0s 81ms/step
vibrato
1/1 [==============================

In [ ]:
len(techniques) # 총 524중 360구간이 저장됐는데 다 vibrato네,, 

360

In [ ]:
print(idx)

524


# Fine Data

In [ ]:
# "belt": 0, "straight": 1, "vibrato": 2, "vocal_fry": 3
sec = [[5,6], [8,9], [13, 14], [16,17], [17,18], [19,20], [25, 26], [27, 28], [28, 29], [30, 31], [32,33], [36,37], [38,39], [47,48], [49,50], [52,53], [58,59],
       [74,75], [79,80], [80,81], [85,86], [89,90], [93, 94],[95,95.5],[95.5,96], [96,97], [98,99], [99,100], [102,103], [104,105], [107,108], [113,114], [115,116],[118,119], [124,125],
       [141,142], [146,147],[148,149],[153,154],[160,160.5],[162,163],[170,171],[173,174]]

pre_sec = [[6.1,6.5],[9.1,9.5],[13.2,13.6],[16.8,17.2],[17.2,17.6],[20.1,20.5],[25.3,25.7],[27.8,28.2],[29.2,29.6],[30.6,31],[32.1,32.5],
           [36.1,36.5],[38.9,39.3],[47.5,47.9],[49.8,50.2],[52.2,52.6],[58.6,59.0],
           [74.6,75.0],[80.0,80.4],[80.3,80.7],[85.6,86.0],[89.8,90.2],[94.1,94.5],[95.4,95.8],[96.0,96.4],[96.9,97.3],[98.2,98.6],[99.5,99.9],[102.5,102.9],[105.1,105.5],[107.0,107.4],[113.7,114.1],[116.0,116.4],[118.4,118.8],[124.8,125.2],
           [141.7,142.1],[146.7,147.1],[148.9,149.3],[153.5,153.9],[160.3,160.7],[162.6,163.0],[171.2,171.6],[173.9,174.3]]
label = ["vocal_fry","vocal_fry","vocal_fry","vocal_fry","vocal_fry","vocal_fry","vibrato","belt",'belt','belt','vibrato','vibrato', 'belt','vibrato','belt','belt','vibrato',
         "vocal_fry","vocal_fry","vibrato","vocal_fry","vocal_fry",'belt','belt',"vocal_fry",'belt','vibrato', "vocal_fry",'belt','belt','belt','vibrato','belt','belt','vibrato',
         "vocal_fry", 'vibrato','vibrato','vibrato',
         'belt','belt','belt','belt']

In [ ]:
# 정확한 0.4초 확인하기 위한 코드
for i in sec[42:]:
  a,b = i[0],i[1]+1
  display(Audio(data = fine[int((a)*44100) : int((b)*44100)], rate = 44100))
  idx = 0
  while a+idx+0.4 <= b:
    print(a+idx)
    print(a+idx+0.4)
    display(Audio(data = fine[int((a+idx)*44100) : int((a+idx+0.4)*44100)], rate = 44100))
    idx+=0.1
  break 

In [ ]:
fine_with_mr = []
for i in pre_sec:
  print(i)
  fine_with_mr.append(fine[int(i[0]*44100) : int(i[1]*44100)])
original_df = pd.DataFrame({"data" : fine_with_mr,"label" : label})  

In [ ]:
# original_df.to_pickle("/content/drive/MyDrive/Startup_Hackathon_Jarvis/fine_data/fine_mr.pkl")  

In [ ]:
len(ft2d_fg.audio_data[0])

9251329

In [ ]:
fine_without_mr = []
for i in pre_sec:
  fine_without_mr.append(ft2d_fg.audio_data[0][int(i[0]*44100) : int(i[1]*44100)])
no_mr_df = pd.DataFrame({"data" : fine_without_mr,"label" : label})  

no_mr_df.to_pickle("/content/drive/MyDrive/Startup_Hackathon_Jarvis/fine_data/fine_vocal.pkl")  

In [ ]:
for i in fine_without_mr:
  display(Audio(data = i, rate = 44100))

# 성능 테스트

In [ ]:
fine_with_mr = pd.read_pickle("/content/drive/MyDrive/Startup_Hackathon_Jarvis/fine_data/fine_mr.pkl")  
fine_without_mr = pd.read_pickle("/content/drive/MyDrive/Startup_Hackathon_Jarvis/fine_data/fine_vocal.pkl")

In [ ]:
model = load_model("/content/drive/MyDrive/Startup_Hackathon_Jarvis/model_weight/custom_vgg16.h5", compile=False)
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=1e-4), metrics=['accuracy'])
# result = model.predict(amp_re)

In [ ]:
def processing(data):
  #data, _ = librosa.magphase(data)
  ret = librosa.feature.melspectrogram(y=data, sr=44100)
  amp = librosa.amplitude_to_db(np.abs(ret))
  # print(amp.shape)
  return amp

In [ ]:
with_mr = []
for i in fine_with_mr.data:
  with_mr.append(processing(i))
with_mr = tf.reshape(with_mr, [-1, with_mr[0].shape[0], with_mr[0].shape[1], 1])

In [ ]:
without_mr = []
for i in fine_without_mr.data:
  without_mr.append(processing(i))
without_mr = tf.reshape(without_mr, [-1, without_mr[0].shape[0], without_mr[0].shape[1], 1])

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
le = LabelEncoder()
le.fit(['belt', 'straight', 'vibrato', 'vocal_fry'])
with_mr_label = to_categorical(le.transform(np.array(fine_with_mr.label)))
without_mr_label = to_categorical(le.transform(np.array(fine_without_mr.label)))

In [ ]:
with_mr

In [ ]:
# custom_vgg16.h5모델 결과
# with mr : loss: 4.2349 - accuracy: 0.2791
# without mr : loss: 3.1548 - accuracy: 0.2093
model.evaluate(with_mr,with_mr_label)
model.evaluate(without_mr, without_mr_label)

2/2 [==============================] - 1s 363ms/step - loss: 3.1548 - accuracy: 0.2093


[3.154789447784424, 0.20930232107639313]